# Code for transforming the base features to SIS+MLR learnt feature

B. Ouyang, J. Wang, T. He, C. J. Bartel, H. Huo, Y. Wang, V. Lacivita, H. Kim and G. Ceder, Synthetic accessibility and stability rules of NASICONs, arXiv, 2021.

In this work, we used SIS+MLR to select the most useful feature from millions of candidate features. The selected features are:

1. feature_1 = $NNaLst^{1/3}+AnionChgStdLst^2$
2. feature_2 = $EWaldSumLst^2\cdot (XWithNaLst\cdot RDiffStdLst)$

This script transforms the raw basic features (in files `RawData/train.csv` and `RawData/test.csv`) into a 2-dimenstional space. We save the transformed features into `RawData/train_2D.csv` and `RawData/test_2D.csv`.

- **Input data with basic features:** RawData/train.csv, RawData/test.csv
- **Output data with 2D SIS features:** RawData/train_2D.csv, RawData/test_2D.csv

## Predefined functions

We will be using `eval()` to evaluate the formula.

In [1]:
import os

import numpy as np
import pandas as pd
from pandas import DataFrame
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score

def __compatFeature(feature):
    '''
    Make the feature compatible to be eval()

    List of operator in SISSO are:
    (+)(-)(*)(/)(exp)(log)(^-1)(^2)(^3)(sqrt)(cbrt)(|-|)
    '''
    if 'exp' in feature: 
        feature = feature.replace('exp','np.exp')
    if 'log' in feature: 
        feature = feature.replace('log','np.log')
    if '^' in feature: 
        feature = feature.replace('^','**')
    if 'sqrt' in feature: 
        feature = feature.replace('sqrt','np.sqrt')
    if 'cbrt' in feature: 
        feature = feature.replace('cbrt','np.cbrt')
    if '-1' in feature: 
        feature = feature.replace('-1','(-1)')
    if '|' in feature:
        all_abs_strs = re.findall(r'\|.*?\|', feature)
        for all_abs_str in all_abs_strs:
            subs_str = 'np.abs({})'.format(all_abs_str.strip('\|\|'))
            feature = feature.replace(all_abs_str, subs_str)
    return feature

def __getFeatureMat(X, base_names, feature_comp):
    '''
    Get feature matrix
    '''
    eval_ns = {name:X[:, ind] for ind, name in enumerate(base_names)}
    feature_mat = [];
    for feature in feature_comp:
        compat_feature = __compatFeature(feature);
        exec(f'result={compat_feature}', None, eval_ns)
        feature_lst = eval_ns['result'].tolist()
        feature_mat.append(feature_lst)
    feature_mat = np.asarray(feature_mat)
    return feature_mat.transpose()

## Feature transformation

### Load basic features

In [2]:
train_df = pd.read_csv('../RawData/train.csv')
test_df = pd.read_csv('../RawData/test.csv')
with open('../RawData/KeyFeatureNames.txt','r') as fid:
    key_feature_names = eval(fid.read())

In [3]:
train_df.head()

,CompName,NewEhullLst,EWaldSumLst,EFormLst,RDiffLst,RDiffStdLst,RDiffNaLst,EleNegStdLst,XDiffLst,AnionXStdLst,XWithNaLst,ChgDiffLst,ChgStdLst,AnionChgStdLst,NNaLst,RRatioNaLst
0,MgZr(SO4)3,-60.260567,-65.921154,-1.891755,0.000,0.000000,-0.300000,0.010000,0.02,0.0,0.390000,2.0,1.000000,0.0,0.0,0.705882
1,MgTi(SO4)3,-27.986572,-66.524285,-1.828661,0.115,0.057500,-0.357500,0.115000,0.23,0.0,0.495000,2.0,1.000000,0.0,0.0,0.649510
2,MgSn(SO4)3,-30.640966,-66.032051,-1.554568,0.030,0.015000,-0.315000,0.325000,0.65,0.0,0.705000,2.0,1.000000,0.0,0.0,0.691176
3,Mg4Nb2(SO4)9,-42.783127,-66.565193,-1.750108,0.080,0.037712,-0.326667,0.136707,0.29,0.0,0.476667,3.0,1.414214,0.0,0.0,0.679739
4,ZrZn(SO4)3,-40.419096,-65.810136,-1.699355,0.020,0.010000,-0.290000,0.160000,0.32,0.0,0.560000,2.0,1.000000,0.0,0.0,0.715686


In [4]:
test_df.head()

,CompName,NewEhullLst,EWaldSumLst,EFormLst,RDiffLst,RDiffStdLst,RDiffNaLst,EleNegStdLst,XDiffLst,AnionXStdLst,XWithNaLst,ChgDiffLst,ChgStdLst,AnionChgStdLst,NNaLst,RRatioNaLst
0,MgGe(SO4)3,-9.215538,-66.427225,-1.520901,0.19,0.095000,-0.395000,0.350000,0.70,0.0,0.730000,2.0,1.000000,0.0,0.0,0.612745
1,Mg4Ta2(SO4)9,-48.215659,-66.611213,-1.785828,0.08,0.037712,-0.326667,0.089567,0.19,0.0,0.443333,3.0,1.414214,0.0,0.0,0.679739
2,ZnSn(SO4)3,-21.229072,-65.840156,-1.350749,0.05,0.025000,-0.305000,0.155000,0.31,0.0,0.875000,2.0,1.000000,0.0,0.0,0.700980
3,ZnGe(SO4)3,-25.113430,-66.215531,-1.285539,0.21,0.105000,-0.385000,0.180000,0.36,0.0,0.900000,2.0,1.000000,0.0,0.0,0.622549
4,CaHf(SO4)3,-105.503851,-65.771901,-1.947371,0.29,0.145000,-0.165000,0.150000,0.30,0.0,0.220000,2.0,1.000000,0.0,0.0,0.838235


In [5]:
print('Key feature names:')
for i, name in enumerate(key_feature_names):
    print('%d\t%s' % (i, name))

Key feature names:
0	NewEhullLst
1	EWaldSumLst
2	EFormLst
3	RDiffLst
4	RDiffStdLst
5	RDiffNaLst
6	EleNegStdLst
7	XDiffLst
8	AnionXStdLst
9	XWithNaLst
10	ChgDiffLst
11	ChgStdLst
12	AnionChgStdLst
13	NNaLst
14	RRatioNaLst


### Transform basic features to 2D SIS features

In [6]:
feature_comp = ['(cbrt(NNaLst)+(AnionChgStdLst)^2)', '((EWaldSumLst)^2*(XWithNaLst*RDiffStdLst))']

In [7]:
X_train, X_test, Y_train, Y_test = np.float32(train_df.to_numpy()[:,2:]), np.float32(test_df.to_numpy()[:,2:]), \
                                   np.float32(train_df.to_numpy()[:,1]), np.float32(test_df.to_numpy()[:,1])

In [8]:
print(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape)

(3104, 14) (777, 14) (3104,) (777,)


In [9]:
# transformed 2D SIS features
train_feature_mat = __getFeatureMat(X_train, key_feature_names[1:], feature_comp)
test_feature_mat = __getFeatureMat(X_test, key_feature_names[1:], feature_comp)

### Display and save 2D SIS features

In [10]:
train_2D_df = DataFrame.from_dict({
    'CompName':train_df['CompName'].to_numpy(),
    'Ehull': Y_train,
    '(cbrt(NNaLst)+(AnionChgStdLst)^2)': train_feature_mat[:,0],
    '((EWaldSumLst)^2*(XWithNaLst*RDiffStdLst))': train_feature_mat[:,1]
})

test_2D_df = DataFrame.from_dict({
    'CompName':test_df['CompName'].to_numpy(),
    'Ehull': Y_test,
    '(cbrt(NNaLst)+(AnionChgStdLst)^2)': test_feature_mat[:,0],
    '((EWaldSumLst)^2*(XWithNaLst*RDiffStdLst))': test_feature_mat[:,1]
})

In [11]:
train_2D_df.head()

,CompName,Ehull,(cbrt(NNaLst)+(AnionChgStdLst)^2),((EWaldSumLst)^2*(XWithNaLst*RDiffStdLst))
0,MgZr(SO4)3,-60.260567,0.0,0.000000
1,MgTi(SO4)3,-27.986572,0.0,125.960243
2,MgSn(SO4)3,-30.640966,0.0,46.109451
3,Mg4Nb2(SO4)9,-42.783127,0.0,79.651306
4,ZrZn(SO4)3,-40.419098,0.0,24.253452


In [12]:
test_2D_df.head()

,CompName,Ehull,(cbrt(NNaLst)+(AnionChgStdLst)^2),((EWaldSumLst)^2*(XWithNaLst*RDiffStdLst))
0,MgGe(SO4)3,-9.215537,0.0,306.012177
1,Mg4Ta2(SO4)9,-48.215660,0.0,74.183754
2,ZnSn(SO4)3,-21.229073,0.0,94.826508
3,ZnGe(SO4)3,-25.113430,0.0,414.334930
4,CaHf(SO4)3,-105.503853,0.0,137.997589


In [13]:
train_2D_df.to_csv('train_2D.csv',index=False)
test_2D_df.to_csv('test_2D.csv',index=False)